In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras import layers, losses, Input
from tensorflow.keras.models import Model

2022-05-12 13:57:51.093073: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Links:

- https://blog.keras.io/building-autoencoders-in-keras.html
- https://www.tensorflow.org/tutorials/generative/autoencoder


In [5]:
pretrain_features = pd.read_csv('handout/pretrain_features.csv')
pretrain_labels = pd.read_csv('handout/pretrain_labels.csv')
train_features = pd.read_csv('handout/train_features.csv')
train_labels = pd.read_csv('handout/train_labels.csv')
test_features = pd.read_csv('handout/test_features.csv')

## Cleaning data

In [6]:
for c in pretrain_features.columns[2:]:
    if len(pretrain_features[c].unique()) < 2:
        pretrain_features = pretrain_features.drop(c, axis=1)
        train_features = train_features.drop(c, axis=1)
        test_features = test_features.drop(c, axis=1)

## Autoencoder

In [7]:
class Autoencoder(Model):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
            Input(shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(latent_dim)
        ])
        self.decoder = tf.keras.Sequential([
            Input(shape=(latent_dim,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [8]:
X_pretrain_train, X_pretrain_val, y_pretrain_train, y_pretrain_val = train_test_split(
    pretrain_features[pretrain_features.columns[2:]], pretrain_labels[pretrain_labels.columns[2:]], test_size=0.1, random_state=42
)


In [9]:
autoencoder = Autoencoder(X_pretrain_train.shape[1], latent_dim=32)
autoencoder.compile(optimizer='adam', loss=losses.MSE)

2022-05-12 13:59:04.041752: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-12 13:59:04.043535: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 13:59:04.043573: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-12 13:59:04.043621: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t3ui01.psi.ch): /proc/driver/nvidia/version does not exist
2022-05-12 13:59:04.044268: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild Ten

In [10]:
history = autoencoder.fit(X_pretrain_train, X_pretrain_train, epochs=10, batch_size=32, validation_data=(X_pretrain_val, X_pretrain_val))

2022-05-12 13:59:07.148559: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-12 13:59:07.149113: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294660000 Hz


Epoch 1/10
1407/1407 [==============================] - 5s 3ms/step - loss: 0.0222 - val_loss: 0.0142
Epoch 2/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0136 - val_loss: 0.0123
Epoch 3/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0119 - val_loss: 0.0112
Epoch 4/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0110 - val_loss: 0.0107
Epoch 5/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0103 - val_loss: 0.0101
Epoch 6/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0099 - val_loss: 0.0098
Epoch 7/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0095 - val_loss: 0.0094
Epoch 8/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0092 - val_loss: 0.0092
Epoch 9/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0090 - val_loss: 0.0090
Epoch 10/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.008

In [75]:
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error

In [67]:
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(train_features[train_features.columns[2:]], train_labels[["homo_lumo_gap"]], test_size=0.05, random_state=42)
X_test = test_features[test_features.columns[2:]].values

encoded_X_train_train = autoencoder.encoder.predict(X_train_train)
encoded_X_train_val = autoencoder.encoder.predict(X_train_val)
encoded_X_test = autoencoder.encoder.predict(X_test)

In [68]:
cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=42)
regressor = LassoCV(alphas=np.linspace(0.00001, 1, 1000), max_iter=10000, cv=cv)

In [73]:
regressor.fit(encoded_X_train_train, y_train_train.values.ravel())

LassoCV(alphas=array([1.00000000e-05, 1.01099099e-03, 2.01198198e-03, 3.01297297e-03,
       4.01396396e-03, 5.01495495e-03, 6.01594595e-03, 7.01693694e-03,
       8.01792793e-03, 9.01891892e-03, 1.00199099e-02, 1.10209009e-02,
       1.20218919e-02, 1.30228829e-02, 1.40238739e-02, 1.50248649e-02,
       1.60258559e-02, 1.70268468e-02, 1.80278378e-02, 1.90288288e-02,
       2.00298198e-02, 2.10308108e-0...
       9.80981171e-01, 9.81982162e-01, 9.82983153e-01, 9.83984144e-01,
       9.84985135e-01, 9.85986126e-01, 9.86987117e-01, 9.87988108e-01,
       9.88989099e-01, 9.89990090e-01, 9.90991081e-01, 9.91992072e-01,
       9.92993063e-01, 9.93994054e-01, 9.94995045e-01, 9.95996036e-01,
       9.96997027e-01, 9.97998018e-01, 9.98999009e-01, 1.00000000e+00]),
        cv=RepeatedKFold(n_repeats=2, n_splits=3, random_state=42),
        max_iter=10000)

In [76]:
y_val_pred = regressor.predict(encoded_X_train_val)

print("Validation MSE:", mean_squared_error(y_train_val.values.ravel(), y_val_pred))

Validation MSE: 0.06832047636717209


In [83]:
y_test_pred = regressor.predict(encoded_X_test)
output_df = pd.DataFrame({"Id": test_features["Id"], "y": y_test_pred})
output_df.to_csv("submission.csv", index=False)